In [1]:
#note requires xlsxwriter

import pandas as pd
from Bio import SeqIO as seqio
import math
import pandas.io.formats.excel
import numpy as np
import requests
import os
from tqdm.notebook import tqdm_notebook
from bisect import bisect_left
from mpire import WorkerPool
from scipy.stats import hypergeom
from statsmodels.stats.multitest import fdrcorrection
import statistics as stat
from scipy.stats import norm




pd.io.formats.excel.ExcelFormatter.header_style = None

In [2]:
# create directory called "out" if it doesn't exist
if not os.path.exists("out"):
    os.makedirs("out")

In [3]:
def download_file(url : str,
                filename : str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    with requests.get(url, headers=headers, stream=True) as response:
        response.raise_for_status()
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk: 
                    file.write(chunk)

def read_matrix_df(scoring_matrices) :
    kin_mat_excel = pd.ExcelFile(scoring_matrices, engine="openpyxl")
    kinase_names = kin_mat_excel.sheet_names
    kin_mat_dfs = {k:kin_mat_excel.parse(k) for k in kinase_names}

    any(d.set_index('AA', inplace=True) for d in kin_mat_dfs.values())
    kin_mat_dicts = {k:d.to_dict() for k,d in kin_mat_dfs.items()}
    kin_mat_dicts = {k:{int(c):aa for c,aa in cl.items()} for k,cl in kin_mat_dicts.items()}
    kin_mat_col = {k:list(d.keys()) for k,d in kin_mat_dicts.items()}
    kin_mat_col = {k:[int(c) for c in cl] for k,cl in kin_mat_col.items()}
    any(c.sort() for c in kin_mat_col.values())

    return (kinase_names, kin_mat_dicts, kin_mat_col)

def score_with_affinity(sequence, col_map, matrix, use_log=False, ignore_mid=True, densitometry=None) :
    mid_res = sequence[5]
    no_mid_seq = sequence[:5] + sequence[6:]
    
    scores = [matrix[col_map[i]][aa] for i,aa in enumerate(no_mid_seq) if (aa != '_')]
    if not ignore_mid and densitometry != None:
        S_S = sum([densitometry[c]['S'] for c in col_map])
        S_T = sum([densitometry[c]['T'] for c in col_map])

        S_ctrl = 0.75 * S_S - 0.25 * S_T
        T_ctrl = 0.75 * S_T - 0.25 * S_S

        S_0 = S_ctrl / (max(S_ctrl, T_ctrl))
        T_0 = T_ctrl / (max(S_ctrl, T_ctrl))
        
        ST_favorability = {'S' : S_0, 'T' : T_0}
        if mid_res not in ST_favorability:
            print('non-S/T mid: %s' %sequence)
            zero_pos = 1
        else: 
            zero_pos = ST_favorability[mid_res]
    else:
        zero_pos = 1

    if use_log :
        scores = [math.log2(s) for s in scores]
        zero_pos = math.log2(zero_pos)
        score = sum(scores) + zero_pos
    else :
        score = math.prod(scores) * zero_pos

    return score

def get_percentile(score, background) :
    i = bisect_left(background, score)
    percentile = i/len(background) * 100
    return percentile

# rearrange PSSM files for easier processing

In [4]:
def rearrange_matrices (matrices_file : str = './out/johnson_matrices.xlsx',
                        sheet_name : str = 'ser_thr_all_norm_scaled_matrice', 
                    output_file : str = './out/ST-Kinases.xlsx'):
    ae_df = pd.read_excel(matrices_file, engine='openpyxl', sheet_name=sheet_name)
    #rename first column to Kinase
    ae_df.rename(columns={ae_df.columns[0]: 'Kinase'}, inplace=True)
    ae_df.set_index('Kinase', inplace=True)

    pos = ['-5', '-4', '-3', '-2', '-1', '1', '2', '3', '4']
    res = ['P','G','A','C','S','T','V','I','L','M','F','Y','W','H','K','R','Q','N','D','E','s','t','y']

    kinase_matrices = {}
    for k,row in ae_df.iterrows() :
        probs = row.to_numpy()
        prob_matrix = np.reshape(probs, (9,23))
        prob_matrix_t = prob_matrix.transpose()
        kdf = pd.DataFrame(prob_matrix_t, columns=pos, index=res)
        kdf.index.name = 'AA'
        kinase_matrices[k] = kdf

    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        any(df.to_excel(writer, sheet_name=k) for k, df in kinase_matrices.items())

In [5]:
johnson_matrices_url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-022-05575-3/MediaObjects/41586_2022_5575_MOESM4_ESM.xlsx'
johnson_matrices_original_file = './out/johnson_matrices.xlsx'

if not os.path.exists(johnson_matrices_original_file):
    download_file(johnson_matrices_url, johnson_matrices_original_file)

johnson_matrices_file = './out/ST-Kinases.xlsx'
rearrange_matrices(output_file=johnson_matrices_file)

densitometry_file = './out/ST-Kinases_densitometry.xlsx'
rearrange_matrices(sheet_name = 'ser_thr_all_raw_matrices', output_file=densitometry_file)

# Create background file using Ochoa et al data

In [6]:
def build_ochoa_background(orginal_background_file : str = './out/ochoa_background.xlsx',
                        sheet_name : str = 'Supplementary Table 3',
                        matrices_file : str = './out/ST-Kinases.xlsx',
                        densitometry_file : str = './out/ST-Kinases_densitometry.xlsx',
                        output_file : str = './out/johnson_ochoa_background_wfav_log.tsv'):
    
    #read the original background file
    #original file is corrupted and needs to be read with calamine instead of openpyxl
    ochoa_df = pd.read_excel(orginal_background_file, engine='calamine', sheet_name=sheet_name)
    ochoa_df = ochoa_df[['Database Uniprot Accession', 'Uniprot Primary Accession', 'Protein', 'Description', 'Phosphosite','SITE_+/-7_AA']]
    ochoa_df['SITE_+4/-5_AA'] = ochoa_df['SITE_+/-7_AA'].apply(lambda x: x[2:-3].upper())
    kinase_names, kin_mat_dicts, kin_mat_col = read_matrix_df(matrices_file)
    _, den_dat_dicts, _ = read_matrix_df(densitometry_file)
    input_df_scores = pd.concat([pd.Series(ochoa_df['SITE_+4/-5_AA'].apply(lambda x: score_with_affinity(x,
        kin_mat_col[k], kin_mat_dicts[k], use_log=False, ignore_mid=False, densitometry = den_dat_dicts[k])),
        index=ochoa_df.index, name=k+'_score') for k in tqdm_notebook(kinase_names)], axis=1)
    input_df_log2 = pd.concat([pd.Series(ochoa_df['SITE_+4/-5_AA'].apply(lambda x: score_with_affinity(x,
                kin_mat_col[k], kin_mat_dicts[k], use_log=True, ignore_mid=False, densitometry = den_dat_dicts[k])),
                index=ochoa_df.index, name=k+'_log2') for k in tqdm_notebook(kinase_names)], axis=1)
    
    ochoa_df = pd.concat([ochoa_df, input_df_scores, input_df_log2], axis=1)

    ochoa_df.to_csv(output_file, sep='\t', index=False)

In [7]:

# original url is corrupted
ochoa_background_url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-022-05575-3/MediaObjects/41586_2022_5575_MOESM5_ESM.xlsx'


ochoa_background_original_file = './out/ochoa_background.xlsx'
ochoa_background_file = './out/johnson_ochoa_background_wfav_log.tsv'


if not os.path.exists(ochoa_background_original_file):
    download_file(ochoa_background_url, ochoa_background_original_file)

if not os.path.exists(ochoa_background_file) :
    #takes a while to run (5+ minutes)
    build_ochoa_background(orginal_background_file=ochoa_background_original_file, matrices_file = johnson_matrices_file, densitometry_file = densitometry_file, output_file=ochoa_background_file)

# Score and peptides of optosos dataset

In [8]:
def score_data_file(data_file : str = './input/siteQuant_optosos10m_fracnofrac_combined_no_scan_scrambling_motifsuniquepaste.xlsx',
                    sheet_name : str = 'Sheet1',
                    matrices_file : str = './out/ST-Kinases.xlsx',
                    densitometry_file : str = './out/ST-Kinases_densitometry.xlsx',
                    background_file : str = './out/johnson_ochoa_background_wfav_log.tsv',
                    output_file : str = './out/siteQuant_optosos10m_fracnofrac_combined_no_scan_scrambling_motifsuniquepaste-all_johnson_ochoa_wfav_log.tsv',
                    skip_log2: bool = False,
                    ranks_only : bool = False,
                    sequence_as_is : bool = False) :
        
        phospho_df = pd.read_excel(data_file, engine='openpyxl', sheet_name=sheet_name)
        if not sequence_as_is:
                phospho_df['Motif'] = phospho_df['Motif'].apply(lambda x: x.replace('x','_'))
                phospho_df['SITE_+4/-5_AA'] = phospho_df['Motif'].apply(lambda x: x[1:-2])
        STonly = phospho_df['SITE_+4/-5_AA'].apply(lambda x: x[5] == 'S' or x[5] == 'T')
        phospho_df = phospho_df[STonly].copy()
        
        kinase_names, kin_mat_dicts, kin_mat_col = read_matrix_df(matrices_file)
        _, den_dat_dicts, _ = read_matrix_df(densitometry_file)
        phospho_df_scores = pd.concat([pd.Series(phospho_df['SITE_+4/-5_AA'].apply(lambda x: score_with_affinity(x,
                kin_mat_col[k], kin_mat_dicts[k], use_log=False, ignore_mid=False, densitometry = den_dat_dicts[k])),
                index=phospho_df.index, name=k+'_score') for k in tqdm_notebook(kinase_names)], axis=1)
        
        if not skip_log2:
                phospho_df_log2 = pd.concat([pd.Series(phospho_df['SITE_+4/-5_AA'].apply(lambda x: score_with_affinity(x,
                        kin_mat_col[k], kin_mat_dicts[k], use_log=True, ignore_mid=False, densitometry = den_dat_dicts[k])),
                        index=phospho_df.index, name=k+'_log2') for k in tqdm_notebook(kinase_names)], axis=1)
    
                phospho_df = pd.concat([phospho_df, phospho_df_scores, phospho_df_log2], axis=1)
        else :

                phospho_df = pd.concat([phospho_df, phospho_df_scores], axis=1)

        background_df = pd.read_csv(background_file, sep='\t')
        background_df = background_df[[k+'_score' for k in kinase_names]].copy()

        kin_dists = {k:list(background_df[k+'_score']) for k in kinase_names}
        any(d.sort() for d in kin_dists.values())

        phospho_df_percentiles = pd.concat([pd.Series(phospho_df[k+'_score'].apply(lambda x: get_percentile(x, kin_dists[k])),
                index=phospho_df.index, name=k+'_percentile') for k in tqdm_notebook(kinase_names)], axis=1)
        phospho_df = pd.concat([phospho_df, phospho_df_percentiles], axis=1)

        if ranks_only:
                phospho_df = pd.concat([phospho_df['SITE_+4/-5_AA'], phospho_df_percentiles], axis=1)
        phospho_df.to_csv(output_file, index=False, sep='\t')


In [9]:
optosos_data = './input/siteQuant_optosos10m_fracnofrac_combined_no_scan_scrambling_motifsuniquepaste.xlsx'

optosos_scored = './out/siteQuant_optosos10m_fracnofrac_combined_no_scan_scrambling_motifsuniquepaste-all_johnson_ochoa_wfav_log.tsv'

if not os.path.exists(optosos_scored):
    #takes a while to run (1+ minute)
    score_data_file(data_file=optosos_data, matrices_file = johnson_matrices_file, densitometry_file = densitometry_file, background_file = ochoa_background_file, output_file=optosos_scored)

# Preprocess for building figure

In [10]:
def binarize(percentile_df, percentile=90) :
  binarized_df = pd.DataFrame(np.where(percentile_df >= percentile, True, False),
  index=percentile_df.index, columns=percentile_df.columns)
  
  binarized_df['Any'] = binarized_df.any(axis=1)
  binarized_df.reset_index(inplace=True, drop=True)
  binarized_df = binarized_df[binarized_df['Any']].copy()
  binarized_df.drop(['Any'], axis=1, inplace=True)
  
  
  return binarized_df

def create_binarize_file(optosos_file : str = './out/siteQuant_optosos10m_fracnofrac_combined_no_scan_scrambling_motifsuniquepaste-all_johnson_ochoa_wfav_log.tsv',
                          fly_orthologs_file : str = './input/uniprotIDs_fly_homologs_non_transmembrane_johnson_kinases.xlsx',
                          output_file : str = './out/siteQuant_optosos10m_binarize.csv') :
    
    cycle_without_membrane = fly_orthologs_file
    cycle_without_membrane_df = pd.read_excel(cycle_without_membrane, engine='openpyxl')
    cycle_without_membrane_df.set_index('Fly Uniprot ID', inplace=True)
    kinases = list(cycle_without_membrane_df['Human kinase name'])

    optosos_10m_df = pd.read_csv(optosos_file, sep='\t')
    optosos_10m_df = optosos_10m_df[optosos_10m_df['Max Score'] >= 13].copy()
    
    experiment_df = optosos_10m_df.filter(regex='_percentile')
    experiment_df.columns = [c.split('_percentile')[0] for c in experiment_df.columns]
    experiment_df = experiment_df[kinases].copy()

    exp_binarize_df = binarize(experiment_df)

    #change all columns from bool to int
    exp_binarize_num_df = exp_binarize_df.astype(int)
    
    exp_binarize_num_df.to_csv(output_file, index=False)
    

In [11]:
fly_orthologs_file : str = './input/uniprotIDs_fly_homologs_non_transmembrane_johnson_kinases.xlsx'
binarize_file = './out/siteQuant_optosos10m_binarize.csv'
create_binarize_file(optosos_file=optosos_scored, fly_orthologs_file=fly_orthologs_file, output_file=binarize_file)

# get background phosphoproteome from expressed genes in early fly

In [12]:
def get_peptide_seqs(seq, left_offset = 5 , right_offset = 4) :
    seq_len = len(seq)
    s_or_t_i = [i for i,aa in enumerate(seq) if (aa == 'S') or (aa == 'T')]
    s_or_t_i = [i for i in s_or_t_i if (i - left_offset >= 0) and (i+right_offset < seq_len)]
    s_or_t_i = [''.join([seq[i-left_offset:i+right_offset+1]]) for i in s_or_t_i]
    s_or_t_i = [s for s in s_or_t_i if ('U' not in s) and ('X' not in s)]
    return s_or_t_i

def build_phosphoproteome(proteome_file : str,
        threads : int = 8,
        output_file : str = './out/early_fly_phosphoproteome.xlsx ') :
    seqrec_dict = seqio.to_dict(seqio.parse(proteome_file, 'fasta'), 
            key_function= lambda x: x.id.split('|')[1])
    seqrec_dict = {k:str(s.seq.upper()) for k,s in seqrec_dict.items()}

    seqs = [list(s) for s in seqrec_dict.values()]
    seqs = [''.join(s).rstrip() for s in seqs]


    with WorkerPool(n_jobs=threads) as executor :
        st_seqs = executor.map(get_peptide_seqs, seqs, progress_bar=True)
    st_seqs = [s for sq in st_seqs for s in sq]
    print(f'Number of 10-mer phosphopeptides : {len(st_seqs)}')
    seq_s = pd.Series(st_seqs, name='SITE_+4/-5_AA')
    seq_df = pd.concat([seq_s], axis=1)

    seq_df.to_excel(output_file, index = False, sheet_name = 'Sheet1', engine = 'xlsxwriter')
    

In [13]:
early_fly_proteome_fasta = './input/early_fly_proteome.fasta'
early_fly_phosphoproteome = './out/early_fly_phosphoproteome.xlsx'
early_fly_phosphoproteome_scored = './out/early_fly_phosphoproteome-all_johnson_ochoa_wfav_ranks.tsv'


if not os.path.exists(early_fly_phosphoproteome):
    build_phosphoproteome(proteome_file=early_fly_proteome_fasta, output_file=early_fly_phosphoproteome, threads = 1)
if not os.path.exists(early_fly_phosphoproteome_scored):
    #takes a while to run (15+ minutes)
    score_data_file(data_file=early_fly_phosphoproteome, matrices_file = johnson_matrices_file, densitometry_file = densitometry_file, background_file = ochoa_background_file, output_file=early_fly_phosphoproteome_scored, skip_log2=True, ranks_only=True, sequence_as_is=True)


# Do enrichment analysis

In [14]:
def kinase_count(percentile_df, percentile=90, normalize=True, binarized=False) :
  if not binarized :
    binarized_df = binarize(percentile_df, percentile)
  else :
    binarized_df = percentile_df.copy()
  total_peptides = len(binarized_df.index)

  binarized_df = binarized_df.transpose()

  counts = binarized_df.sum(axis=1)
  if(normalize) :
    counts = np.divide(counts, total_peptides)
  counts_s = pd.Series(counts, index=binarized_df.index, name='c%d' % (percentile))
  return counts_s

def do_enrichment_analysis(experiment_df, background_df, fdr=0.1) :
  exp_binarize_df = binarize(experiment_df)
  num_experiment_peptides = len(exp_binarize_df.index)

  bak_binarize_df = binarize(background_df)
  num_background_peptides = len(bak_binarize_df.index)

  experiment_count = kinase_count(exp_binarize_df, normalize=False, binarized=True)
  experiment_count.rename('experiment_count',inplace=True)

  background_count = kinase_count(bak_binarize_df, normalize=False, binarized=True)
  background_count.rename('background_count',inplace=True)

  experiment_enrich = kinase_count(exp_binarize_df, binarized=True)
  experiment_enrich.rename('experiment_ratio',inplace=True)

  background_enrich = kinase_count(bak_binarize_df, binarized=True)
  background_enrich.rename('background_ratio',inplace=True)

  count_df = pd.concat([background_count, experiment_count, background_enrich, experiment_enrich], axis=1)
  count_df['enrichment'] = np.divide(count_df['experiment_ratio'], count_df['background_ratio'])

  count_df['cdf'] = hypergeom.cdf(k = count_df['experiment_count'],
                  M = num_background_peptides,
                  n = count_df['background_count'],
                  N = num_experiment_peptides)

  count_df['pmf'] = hypergeom.pmf(k = count_df['experiment_count'],
                  M = num_background_peptides,
                  n = count_df['background_count'],
                  N = num_experiment_peptides)
  
  count_df['p-value'] = count_df.apply(lambda x: 1 - x['cdf'] + x['pmf'] if x['enrichment'] >= 1.0 else x['cdf'], axis=1)

  count_df['fdr'] = fdrcorrection(count_df['p-value'], alpha=fdr)[1]

  count_df.drop(['cdf', 'pmf'], axis=1, inplace=True)

  count_df.sort_values(by=['enrichment'], ascending=False, inplace=True)


  return (num_experiment_peptides, num_background_peptides, count_df)

In [15]:
enrichment_dfs = {}
peptide_counts = []
fdr_cutoffs = {'topright_vs_all' : 20, 'all_vs_proteome' : 0.10}

cycle_without_membrane = fly_orthologs_file
cycle_without_membrane_df = pd.read_excel(cycle_without_membrane, engine='openpyxl')
cycle_without_membrane_df.set_index('Fly Uniprot ID', inplace=True)
kinases = list(cycle_without_membrane_df['Human kinase name'])

name = 'topright_vs_all'
optosos_10m_df = pd.read_csv(optosos_scored, sep='\t')
optosos_10m_df = optosos_10m_df[optosos_10m_df['Max Score'] >= 13].copy()
optosos_10m_top_right_df = optosos_10m_df[(optosos_10m_df['neglogadjp'] >= 2) & (optosos_10m_df['log2FC'] >= 0)].copy()

optosos_10m_df  = optosos_10m_df.filter(regex='_percentile')
optosos_10m_df.columns = [c.split('_percentile')[0] for c in optosos_10m_df .columns]
optosos_10m_df = optosos_10m_df[kinases].copy()

optosos_10m_top_right_df  = optosos_10m_top_right_df.filter(regex='_percentile')
optosos_10m_top_right_df.columns = [c.split('_percentile')[0] for c in optosos_10m_top_right_df.columns]
optosos_10m_top_right_df = optosos_10m_top_right_df[kinases].copy()


(num_optosos_10m_top_right_peptides, num_optosos_10m_peptides, optosos_10m_topright_vs_all_df) = do_enrichment_analysis(optosos_10m_top_right_df, optosos_10m_df, fdr=fdr_cutoffs[name])

del optosos_10m_top_right_df

#optosos_10m_topright_vs_all_df.index.name = 'kinase'
#enrichment_dfs[name] = optosos_10m_topright_vs_all_df
#peptide_counts.append((name, num_optosos_10m_top_right_peptides, num_optosos_10m_peptides, fdr_cutoffs[name]))

name = 'all_vs_proteome'

early_fly_phosphoproteome_df = pd.read_csv(early_fly_phosphoproteome_scored, sep='\t')
early_fly_phosphoproteome_df.drop(list(early_fly_phosphoproteome_df.filter(regex='_score|_log2|SITE_')), axis=1, inplace=True)
early_fly_phosphoproteome_df.columns = [c.split('_percentile')[0] for c in early_fly_phosphoproteome_df.columns]
early_fly_phosphoproteome_df = early_fly_phosphoproteome_df[kinases].copy()

(num_optosos_10m_peptides, num_early_fly_proteome_peptides, optosos_10m_all_vs_proteome_df) = do_enrichment_analysis(optosos_10m_df, early_fly_phosphoproteome_df, fdr=fdr_cutoffs[name])

del early_fly_phosphoproteome_df
del optosos_10m_df

optosos_10m_all_vs_proteome_df.index.name = 'kinase'
enrichment_dfs[name] = optosos_10m_all_vs_proteome_df
peptide_counts.append((name, num_optosos_10m_peptides, num_early_fly_proteome_peptides, fdr_cutoffs[name]))

# Add summary sheet
summary_counts_df = pd.DataFrame(peptide_counts, columns=['name', 'num experiment peptides', 'num background peptides', 'fdr threshold'])
summary_counts_df.index.name = 'experiment number'
sheets_dict = {'summary': summary_counts_df}
sheets_dict.update(enrichment_dfs)

#combine sheets in Excel file
excel_output = './out/enrichment_variations.xlsx'
with pd.ExcelWriter(excel_output, engine='xlsxwriter') as writer:
  any(df.to_excel(writer, sheet_name=k, index=True) for k, df in sheets_dict.items())


# Z-score analysis

In [16]:
#experiment file
optosos_10m_df = pd.read_csv(optosos_scored, sep='\t')
optosos_10m_df = optosos_10m_df[optosos_10m_df['Max Score'] >= 13].copy()

#56 fly homologs cytoplasmic kinases
cycle_without_membrane = fly_orthologs_file
cycle_without_membrane_df = pd.read_excel(cycle_without_membrane, engine='openpyxl')
cycle_without_membrane_df.set_index('Fly Uniprot ID', inplace=True)
kinases = list(cycle_without_membrane_df['Human kinase name'])

experiment_df = optosos_10m_df.filter(regex='log2FC|_percentile')

experiment_df.columns = [c.split('_percentile')[0] if '_percentile' in c else c for c in experiment_df.columns]

select_columns = ['log2FC']
select_columns.extend(kinases)
experiment_df = experiment_df[select_columns].copy()

pop_mean = stat.mean(experiment_df['log2FC'])
pop_stdv = stat.stdev(experiment_df['log2FC'])

kin_stats = []
for k in kinases :
    k_sub = experiment_df[(experiment_df[k] >= 90)]['log2FC'].copy()
    k_mean = stat.mean(k_sub)
    k_stdv = stat.stdev(k_sub)
    k_n = len(k_sub.index)
    z = (k_mean - pop_mean) / (pop_stdv / math.sqrt(k_n))
    if z > 0 :
        p = 1 - norm.cdf(z) + norm.pdf(z)
    else :
        p = norm.cdf(z)
    #print('%s\t%1.3f\t%0.3e' %(k,z,p))
    kin_stats.append((k, k_n, k_mean, k_stdv, z, p))

pop_stats = [(pop_mean, pop_stdv)]

pop_df = pd.DataFrame(pop_stats, columns=['mean', 'stdv'])
kin_df = pd.DataFrame(kin_stats, columns=['kinase', 'n', 'mean', 'stdv', 'z', 'p-value'])
kin_df['fdr'] = fdrcorrection(kin_df['p-value'])[1]

excel_output = './out/kinase_z_scores_flip.xlsx'
with pd.ExcelWriter(excel_output, engine='xlsxwriter') as writer:
  pop_df.to_excel(writer, sheet_name='population', index=False)
  kin_df.to_excel(writer, sheet_name='kinases', index=False)